In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/netflix-shows/netflix_titles.csv


In [2]:
df = pd.read_csv('C:\Users\ronit\OneDrive\Desktop\College\Sem_7\NNDL\NNDL_Repo\DL_MiniProject\datasets\netflix_titles.csv')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [3]:
df.shape

(8807, 12)

In [4]:
df.isna().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

In [5]:
df1 = df.drop(['show_id', 'type', 'director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'duration'], axis=1)
df1.head()

,title,listed_in,description
0,Dick Johnson Is Dead,Documentaries,"As her father nears the end of his life, filmm..."
1,Blood & Water,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,Ganglands,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,Jailbirds New Orleans,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,Kota Factory,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [ ]:
## All Libraries

# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from transformers import T5ForConditionalGeneration, T5Tokenizer, get_linear_schedule_with_warmup
# import torch
# from torch.utils.data import Dataset, DataLoader
# from tqdm import tqdm
# import re
# import nltk
# from nltk.corpus import stopwords
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

In [6]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import nltk
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
import string
# Preprocessing function to clean the text
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation]
    return ' '.join(tokens)

In [8]:
# Preprocess the description and title columns
df1['description_clean'] = df1['description'].apply(preprocess_text)
df1['title_clean'] = df1['title'].apply(preprocess_text)

In [9]:
df1.head()

,title,listed_in,description,description_clean,title_clean
0,Dick Johnson Is Dead,Documentaries,"As her father nears the end of his life, filmm...",father nears end life filmmaker kirsten johnso...,dick johnson dead
1,Blood & Water,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",crossing paths party cape town teen sets prove...,blood water
2,Ganglands,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,protect family powerful drug lord skilled thie...,ganglands
3,Jailbirds New Orleans,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",feuds flirtations toilet talk go among incarce...,jailbirds new orleans
4,Kota Factory,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,city coaching centers known train india ’ fine...,kota factory


In [10]:
from torch.utils.data import Dataset, DataLoader
# Create a custom dataset
class NetflixDataset(Dataset):
    def __init__(self, descriptions, titles, tokenizer, max_length):
        self.descriptions = descriptions
        self.titles = titles
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.descriptions)

    def __getitem__(self, idx):
        if isinstance(idx, list):
            return self.get_batch(idx)
        
        description = self.descriptions[idx]
        title = self.titles[idx]

        return self.process_item(description, title)

    def get_batch(self, indices):
        descriptions = [self.descriptions[i] for i in indices]
        titles = [self.titles[i] for i in indices]

        input_encodings = self.tokenizer(
            ["summarize: " + desc for desc in descriptions],
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        target_encodings = self.tokenizer(
            titles,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        return {
            "input_ids": input_encodings.input_ids,
            "attention_mask": input_encodings.attention_mask,
            "labels": target_encodings.input_ids
        }

    def process_item(self, description, title):
        input_encoding = self.tokenizer(
            "summarize: " + description,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        target_encoding = self.tokenizer(
            title,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        return {
            "input_ids": input_encoding.input_ids.squeeze(),
            "attention_mask": input_encoding.attention_mask.squeeze(),
            "labels": target_encoding.input_ids.squeeze()
        }


In [11]:
from sklearn.model_selection import train_test_split
# Split the data
train_df, val_df = train_test_split(df1, test_size=0.1, random_state=42)

In [12]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
# Load T5 model and tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [13]:
from torch.utils.data import Dataset, DataLoader
# Create datasets and dataloaders
max_length = 128
train_dataset = NetflixDataset(train_df['description_clean'].tolist(), train_df['title_clean'].tolist(), tokenizer, max_length)
val_dataset = NetflixDataset(val_df['description_clean'].tolist(), val_df['title_clean'].tolist(), tokenizer, max_length)

batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

In [15]:
import torch
from transformers import get_linear_schedule_with_warmup
# Training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
num_epochs = 5
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

In [16]:
# Early stopping setup
best_val_loss = float('inf')
patience = 3
no_improve_epochs = 0

In [17]:
from tqdm import tqdm
# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1} - Average training loss: {avg_train_loss:.4f}")

    # Validation
    model.eval()
    total_val_loss = 0

    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc="Validation"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(val_dataloader)
    print(f"Epoch {epoch + 1} - Average validation loss: {avg_val_loss:.4f}")

    # Early stopping check
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        no_improve_epochs = 0
        # Save the best model
        torch.save(model.state_dict(), "best_netflix_title_generator.pth")
    else:
        no_improve_epochs += 1

    if no_improve_epochs >= patience:
        print(f"Early stopping triggered after {epoch + 1} epochs")
        break

Epoch 1: 100%|██████████| 496/496 [04:19<00:00,  1.91it/s]


Epoch 1 - Average training loss: 0.6256


Validation: 100%|██████████| 56/56 [00:09<00:00,  6.12it/s]


Epoch 1 - Average validation loss: 0.1833


Epoch 2: 100%|██████████| 496/496 [04:18<00:00,  1.92it/s]


...

Epoch 4 - Average validation loss: 0.1661


Epoch 5: 100%|██████████| 496/496 [04:18<00:00,  1.92it/s]


Epoch 5 - Average training loss: 0.1703


Validation: 100%|██████████| 56/56 [00:09<00:00,  6.11it/s]


Epoch 5 - Average validation loss: 0.1654


In [18]:
# Load the best model
model.load_state_dict(torch.load("best_netflix_title_generator.pth"))

/tmp/ipykernel_30/2675779898.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_netflix_title_generator.pth"))


<All keys matched successfully>

In [139]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# # New function to select the best title
# def select_best_title(description, titles):
#     # Combine description and titles into a single list
#     texts = [description] + titles
    
#     # Create TF-IDF vectorizer
#     vectorizer = TfidfVectorizer()
#     tfidf_matrix = vectorizer.fit_transform(texts)
    
#     # Calculate cosine similarity between description and each title
#     cosine_similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()
    
#     # Find the index of the title with highest similarity
#     best_index = cosine_similarities.argmax()
    
#     return titles[best_index]

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def capitalize_title(title):
    """Capitalize the first letter of each word in the title."""
    return ' '.join(word.capitalize() for word in title.split())

# Updated function to generate titles
def generate_title(description, model, tokenizer, max_length=20, num_return_sequences=5):
    model.eval()
    input_text = "summarize: " + description
    input_ids = tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True).input_ids.to(device)

    outputs = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        num_beams=num_return_sequences,
        no_repeat_ngram_size=2,
        early_stopping=True
    )

    generated_titles = [capitalize_title(tokenizer.decode(output, skip_special_tokens=True)) for output in outputs]
    return generated_titles

# Function to select the best title (remains the same)
def select_best_title(description, titles):
    texts = [description] + titles
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(texts)
    cosine_similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()
    best_index = cosine_similarities.argmax()
    return titles[best_index]

In [22]:
# Example usage
sample_descriptions = [
    "When his son goes missing during a snowy hike in the mountains, a retired intelligence officer will stop at nothing — and risk everything — to find him.",
    "After a deadly home invasion at a couple's new house, the traumatized wife searches for answers and learns the real danger is just beginning."
]

for description in sample_descriptions:
    generated_titles = generate_title(description, model, tokenizer)
    best_title = select_best_title(description, generated_titles)
    
    print(f"Description: {description}")
    print("Generated Titles:")
    for i, title in enumerate(generated_titles, 1):
        print(f"{i}. {title}")
    print(f"\nBest Generated Title: {best_title}")
    print()

Description: When his son goes missing during a snowy hike in the mountains, a retired intelligence officer will stop at nothing — and risk everything — to find him.
Generated Titles:
1. Son Goes Missing During A Snowy Hike
2. Son Goes Missing During Hike In The Mountains
3. Son Goes Missing During Snowy Hike
4. Son Goes Missing
5. Son

Best Generated Title: Son Goes Missing During Hike In The Mountains

Description: After a deadly home invasion at a couple's new house, the traumatized wife searches for answers and learns the real danger is just beginning.
Generated Titles:
1. Home Invasion
2. 's Home
3. 's House
4. 's Home Invasion
5. Home

Best Generated Title: Home Invasion



In [26]:
# Example usage
sample_descriptions = [
    "Struggling to earn a living in Bangkok, a man joins an emergency rescue service and realizes he must unravel a Citywide Conspiracy.",
    "A father's Christmas Eve turns into a scramble to find a much sought-after Toy for his Son."
]

for description in sample_descriptions:
    generated_titles = generate_title(description, model, tokenizer)
    best_title = select_best_title(description, generated_titles)
    
    print(f"Description: {description}")
    print("Generated Titles:")
    for i, title in enumerate(generated_titles, 1):
        print(f"{i}. {title}")
    print(f"\nBest Generated Title: {best_title}")
    print()

Description: Struggling to earn a living in Bangkok, a man joins an emergency rescue service and realizes he must unravel a Citywide Conspiracy.
Generated Titles:
1. A Man's Life In Bangkok
2. A Man's Life In The City
3. Aman
4. Rescuer
5. Bangkok

Best Generated Title: A Man's Life In Bangkok

Description: A father's Christmas Eve turns into a scramble to find a much sought-after Toy for his Son.
Generated Titles:
1. Christmas Toy
2. Christmas Tree
3. Toy
4. Christmas
5. Christmas Eve

Best Generated Title: Christmas Eve



In [33]:
# Example usage
sample_descriptions = [
    "This contemporary romantic comedy love based on a global bestseller follows native New Yorker Bai Lu to Singapore to meet her boyfriend's family.",
    "Chicago Party Aunt Diane is an idolized troublemaker with a talent for avoiding adulthood â€” and a soft spot for her soul-searching nephew."
]

for description in sample_descriptions:
    generated_titles = generate_title(description, model, tokenizer)
    best_title = select_best_title(description, generated_titles)
    
    print(f"Description: {description}")
    print("Generated Titles:")
    for i, title in enumerate(generated_titles, 1):
        print(f"{i}. {title}")
    print(f"\nBest Generated Title: {best_title}")
    print()

Description: This contemporary romantic comedy love based on a global bestseller follows native New Yorker Bai Lu to Singapore to meet her boyfriend's family.
Generated Titles:
1. Love
2. Love Love
3. Bai Lu Love
4. Romantic Love
5. Bai Lu

Best Generated Title: Bai Lu Love

Description: Chicago Party Aunt Diane is an idolized troublemaker with a talent for avoiding adulthood â€” and a soft spot for her soul-searching nephew.
Generated Titles:
1. 's Life
2. Aunt Diane
3. 's Daughter
4. 's Soul
5. 's Ain'

Best Generated Title: Aunt Diane



In [37]:
# Example usage
sample_descriptions = [
    "Natasha confronts her history as a spy and the broken relationships left in her wake.",
    "A Metro Family decides to fight a Cyber Criminal threatening their stability and pride."
]

for description in sample_descriptions:
    generated_titles = generate_title(description, model, tokenizer)
    best_title = select_best_title(description, generated_titles)
    
    print(f"Description: {description}")
    print("Generated Titles:")
    for i, title in enumerate(generated_titles, 1):
        print(f"{i}. {title}")
    print(f"\nBest Generated Title: {best_title}")
    print()

Description: Natasha confronts her history as a spy and the broken relationships left in her wake.
Generated Titles:
1. Natasha
2. Natasha's Secrets
3. Natasha's Life
4. Natasha Spies
5. Natasha's Story

Best Generated Title: Natasha

Description: A Metro Family decides to fight a Cyber Criminal threatening their stability and pride.
Generated Titles:
1. Metro Family
2. A Cyber Criminal
3. Metro Family
4. Cyber Criminal
5. 's Life

Best Generated Title: Metro Family



In [39]:
# Save the model and tokenizer
output_dir = "/kaggle/working/my_current_model"  # Directory where the model will be saved

# Create the directory if it doesn't exist
import os
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the model's state dictionary
model.save_pretrained(output_dir)

# Save the tokenizer
tokenizer.save_pretrained(output_dir)

print(f"Model and tokenizer saved to {output_dir}")

Model and tokenizer saved to /kaggle/working/my_current_model


In [145]:
# Example usage
sample_descriptions = [
    "When his son goes missing during a snowy hike in the mountains, a retired intelligence officer will stop at nothing — and risk everything — to find him.",
    "After a deadly home invasion at a couple's new dream house, the traumatized wife searches for answers and learns the real danger is just beginning."
]

for description in sample_descriptions:
    generated_titles = generate_title(description, model, tokenizer)
    best_title = select_best_title(description, generated_titles)
    
    print(f"Description: {description}")
    print("Generated Titles:")
    for i, title in enumerate(generated_titles, 1):
        print(f"{i}. {title}")
    print(f"\nBest Generated Title: {best_title}")
    print()

Description: When his son goes missing during a snowy hike in the mountains, a retired intelligence officer will stop at nothing — and risk everything — to find him.
Generated Titles:
1. El Paso Del Cielo
2. Saab
3. Missing Boy
4. Missing Son
5. Son

Best Generated Title: Missing Son

Description: After a deadly home invasion at a couple's new dream house, the traumatized wife searches for answers and learns the real danger is just beginning.
Generated Titles:
1. Home Invasion
2. 's Home
3. 's House
4. House Fire
5. Adrift

Best Generated Title: Home Invasion



In [148]:
# Example usage
sample_descriptions = [
    "The female lead confronts her history as a spy and the broken relationships left in her wake.",
    "A Metro Family decides to fight a Cyber Criminal threatening their stability and pride."
]

for description in sample_descriptions:
    generated_titles = generate_title(description, model, tokenizer)
    best_title = select_best_title(description, generated_titles)
    
    print(f"Description: {description}")
    print("Generated Titles:")
    for i, title in enumerate(generated_titles, 1):
        print(f"{i}. {title}")
    print(f"\nBest Generated Title: {best_title}")
    print()

Description: The female lead confronts her history as a spy and the broken relationships left in her wake.
Generated Titles:
1. Spy's Wife
2. Spy's Tale
3. Spy
4. Spies
5. Spy Woman

Best Generated Title: Spy

Description: A Metro Family decides to fight a Cyber Criminal threatening their stability and pride.
Generated Titles:
1. Adrift
2. Metro
3. Adolescent
4. Metro Family
5. Metro

Best Generated Title: Metro Family



In [149]:
# Example usage
sample_descriptions = [
    "This contemporary romantic comedy based on a global bestseller follows native New Yorker Bai Lu to Singapore to meet her boyfriend's family.",
    "Chicago Party Aunt Diane is an idolized troublemaker with a talent for avoiding adulthood â€” and a soft spot for her soul-searching nephew."
]

for description in sample_descriptions:
    generated_titles = generate_title(description, model, tokenizer)
    best_title = select_best_title(description, generated_titles)
    
    print(f"Description: {description}")
    print("Generated Titles:")
    for i, title in enumerate(generated_titles, 1):
        print(f"{i}. {title}")
    print(f"\nBest Generated Title: {best_title}")
    print()

Description: This contemporary romantic comedy based on a global bestseller follows native New Yorker Bai Lu to Singapore to meet her boyfriend's family.
Generated Titles:
1. 's Got Ta
2. Bai Lu's Love Story
3. 's Love
4. 's Ain't It
5. 's Gonna Scream

Best Generated Title: Bai Lu's Love Story

Description: Chicago Party Aunt Diane is an idolized troublemaker with a talent for avoiding adulthood â€” and a soft spot for her soul-searching nephew.
Generated Titles:
1. Aunt Diane
2. 's Ain't
3. Aunt Diane
4. Adrift
5. 's

Best Generated Title: Aunt Diane



In [ ]:
# Save the model and tokenizer
output_dir = "./saved_model"  # Directory where the model will be saved

# Create the directory if it doesn't exist
import os
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the model's state dictionary
model.save_pretrained(output_dir)

# Save the tokenizer
tokenizer.save_pretrained(output_dir)

print(f"Model and tokenizer saved to {output_dir}")
